In [1]:
# ALL imports
import pysam   # for VCF handling
import cyvcf2 # for VCF handling
from cyvcf2 import VCF # for VCF handling
import numpy as np  
import pandas as pd 

# extract EUR samples list names (168000)
onemil = pd.read_csv('synth_data/1_mil/all_chr.txt',  index_col=0)

-----------------------------------------------
## __PIPELINE TO FORMAT THE 100K SAMPLE DATASET__ ##

In [8]:
# get number of controls and cases
samples = list(onemil.columns)[1:]
phenos = onemil.iloc[147]
cases = []
controls = []
for sample in samples:
    if phenos[sample] == '0':
        controls.append(sample)
    else: cases.append(sample)
len(cases), len(controls)

(50250, 117750)

In [19]:
# separate controls from cases 
controls_df = []
controls_df.append(onemil['ID'])
for sample in controls:
   controls_df.append(onemil[sample])
controls_df = pd.DataFrame(controls_df)
controls_df.to_csv('synth_data/1_mil/controls.txt') # save info of all control samples
controls_df

,0,1,2,3,4,5,6,7,8,9,...,138,139,140,141,142,143,144,145,146,147
ID,rs12117229,rs7547573,rs10917536,rs4652846,rs3816989,rs2863210,rs11805303,rs2201841,rs11209026,rs4656077,...,rs6073286,rs259956,rs3810483,rs1297265,rs1009164,rs406418,rs5996763,rs7290488,rs2413583,PHENO
syn504001_syn504001,0/1,1/1,0/1,0/1,0/0,0/1,0/1,1/1,0/0,0/0,...,1/1,0/0,0/1,1/1,0/1,0/0,0/1,0/1,0/0,0
syn504002_syn504002,0/1,0/0,1/1,0/1,0/0,0/0,0/0,0/0,0/0,0/1,...,0/0,0/1,1/1,1/1,0/1,0/0,0/1,0/0,0/0,0
syn504003_syn504003,0/1,0/1,0/0,0/1,1/1,0/0,1/1,1/1,0/0,0/1,...,0/0,0/0,0/1,1/1,0/1,0/0,0/1,0/1,0/0,0
syn504006_syn504006,0/1,0/1,0/0,0/1,0/0,0/0,1/1,1/1,0/0,0/1,...,0/0,0/1,0/1,1/1,0/0,0/1,0/0,0/1,0/0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
syn671994_syn671994,0/1,0/1,0/1,1/1,0/0,0/1,0/0,0/0,0/0,1/1,...,0/0,0/0,0/1,0/0,0/1,0/1,0/1,0/0,0/0,0
syn671995_syn671995,1/1,1/1,0/0,0/1,0/0,1/1,0/0,0/0,0/0,0/1,...,0/0,0/0,1/1,0/1,1/1,1/1,0/0,0/0,0/1,0
syn671997_syn671997,1/1,1/1,0/1,0/0,0/0,0/0,0/1,0/1,0/0,1/1,...,0/0,0/1,0/1,0/0,0/1,0/1,0/1,0/0,0/0,0
syn671999_syn671999,0/1,1/1,0/0,1/1,0/0,1/1,0/1,0/1,0/0,0/1,...,0/1,1/1,0/1,0/1,0/1,0/0,0/0,0/0,0/1,0


In [18]:
# separate cases from controls 
case_df = []
case_df.append(onemil['ID'])
for sample in cases:
   case_df.append(onemil[sample])
case_df = pd.DataFrame(case_df)
case_df.to_csv('synth_data/1_mil/cases.txt') # save info of all case samples
case_df

,0,1,2,3,4,5,6,7,8,9,...,138,139,140,141,142,143,144,145,146,147
ID,rs12117229,rs7547573,rs10917536,rs4652846,rs3816989,rs2863210,rs11805303,rs2201841,rs11209026,rs4656077,...,rs6073286,rs259956,rs3810483,rs1297265,rs1009164,rs406418,rs5996763,rs7290488,rs2413583,PHENO
syn504004_syn504004,1/1,0/1,0/1,0/1,0/0,1/1,0/0,0/0,0/0,0/1,...,0/1,0/1,0/1,0/0,0/1,0/1,0/0,0/0,0/1,1
syn504005_syn504005,0/1,1/1,0/1,0/0,1/1,1/1,0/0,0/0,0/0,0/0,...,0/0,0/1,1/1,0/1,0/1,0/1,0/0,1/1,0/1,1
syn504024_syn504024,0/0,0/1,0/0,0/0,0/0,0/1,0/0,0/0,0/0,0/1,...,0/0,0/1,0/1,0/1,0/0,0/0,0/1,0/0,0/0,1
syn504025_syn504025,0/1,0/1,0/1,0/1,0/0,0/0,0/0,0/1,0/0,0/0,...,0/1,0/0,0/1,0/1,0/1,0/1,0/0,0/0,0/0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
syn671982_syn671982,0/0,0/0,0/0,0/1,0/1,1/1,0/0,0/0,0/1,0/0,...,0/1,1/1,0/1,0/1,0/0,0/1,1/1,0/0,0/0,1
syn671984_syn671984,0/1,1/1,0/1,0/0,0/0,0/1,0/0,0/0,0/0,1/1,...,0/0,0/1,0/1,0/0,0/1,0/0,0/1,0/0,0/0,1
syn671986_syn671986,0/1,0/1,0/1,0/1,0/0,0/1,0/0,0/0,0/0,0/1,...,0/0,1/1,1/1,0/0,0/1,0/1,0/0,0/0,0/0,1
syn671996_syn671996,0/0,0/1,0/0,1/1,0/1,0/1,0/1,0/1,0/1,0/1,...,0/0,1/1,0/1,0/1,1/1,0/0,0/0,0/0,0/1,1


In [25]:
# build custom case-control datasets for neural network training
# can adjust case:sample ratio, but here 50:50 ratio is chosen
train_set = []
train_set.append(onemil['ID'])
# sample_size = 2500 # total 5000 samples
# sample_size = 5000 # total 10000 samples
sample_size = len(cases) # total 1000000+ samples
for sample in cases[:sample_size]:
    train_set.append(onemil[sample])
for sample in controls[:sample_size]:
    train_set.append(onemil[sample])
train_ds = pd.DataFrame(train_set)
train_ds.to_csv('synth_data/1_mil/100k.txt')
train_ds

,0,1,2,3,4,5,6,7,8,9,...,138,139,140,141,142,143,144,145,146,147
ID,rs12117229,rs7547573,rs10917536,rs4652846,rs3816989,rs2863210,rs11805303,rs2201841,rs11209026,rs4656077,...,rs6073286,rs259956,rs3810483,rs1297265,rs1009164,rs406418,rs5996763,rs7290488,rs2413583,PHENO
syn504004_syn504004,1/1,0/1,0/1,0/1,0/0,1/1,0/0,0/0,0/0,0/1,...,0/1,0/1,0/1,0/0,0/1,0/1,0/0,0/0,0/1,1
syn504005_syn504005,0/1,1/1,0/1,0/0,1/1,1/1,0/0,0/0,0/0,0/0,...,0/0,0/1,1/1,0/1,0/1,0/1,0/0,1/1,0/1,1
syn504024_syn504024,0/0,0/1,0/0,0/0,0/0,0/1,0/0,0/0,0/0,0/1,...,0/0,0/1,0/1,0/1,0/0,0/0,0/1,0/0,0/0,1
syn504025_syn504025,0/1,0/1,0/1,0/1,0/0,0/0,0/0,0/1,0/0,0/0,...,0/1,0/0,0/1,0/1,0/1,0/1,0/0,0/0,0/0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
syn575665_syn575665,0/1,1/1,0/0,0/0,0/0,0/0,1/1,1/1,0/0,0/1,...,0/0,0/1,0/1,0/0,0/0,0/1,0/0,0/1,0/0,0
syn575666_syn575666,0/1,0/1,0/0,0/0,0/0,0/0,0/1,0/0,0/0,1/1,...,0/1,1/1,0/1,0/1,0/1,0/0,0/1,0/0,0/0,0
syn575667_syn575667,0/1,0/1,0/0,0/1,0/0,0/1,0/1,0/1,0/0,0/1,...,0/0,0/0,1/1,0/1,0/1,0/1,1/1,1/1,0/1,0
syn575669_syn575669,0/0,0/1,0/1,0/1,0/0,0/0,0/0,0/0,0/0,0/1,...,0/0,0/1,0/1,0/1,0/1,0/0,0/1,0/0,0/1,0


In [18]:
# Calculate manual PRS for each sample 
all_ds = pd.read_csv('/Users/asiabelfiore/Desktop/project/synth_data/1_mil/100k.txt')
liftover = pd.read_csv('data/mutations/rsid_causal/causal_grCh_liftover.txt', sep='\t').set_index('RSID')
mutations = list(all_ds.columns)[1:148]
prs_per_sample = {}
for i in range(all_ds.shape[0]):
    row = all_ds.iloc[i]
    prs = 0
    sample = row['SAMPLE']
    for rsid in mutations:
        x_gt = str(row[rsid]).split('/')
        alt_freq = int(x_gt[0])+int(x_gt[1])
        prs += alt_freq*liftover.loc[rsid]['BETA']
    prs_per_sample[sample] = prs
    if i%1000==0: print(i,' done')

outfile = open('/Users/asiabelfiore/Desktop/per_sample_prs.txt', 'w')
outfile.write('SAMPLE,PRS')
for sample in list(prs_per_sample.keys()):
    outfile.write('\n'+sample+','+str(prs_per_sample[sample]))
outfile.close()

0  done
1000  done
2000  done
3000  done
4000  done
5000  done
6000  done
7000  done
8000  done
9000  done
10000  done
11000  done
12000  done
13000  done
14000  done
15000  done
16000  done
17000  done
18000  done
19000  done
20000  done
21000  done
22000  done
23000  done
24000  done
25000  done
26000  done
27000  done
28000  done
29000  done
30000  done
31000  done
32000  done
33000  done
34000  done
35000  done
36000  done
37000  done
38000  done
39000  done
40000  done
41000  done
42000  done
43000  done
44000  done
45000  done
46000  done
47000  done
48000  done
49000  done
50000  done
51000  done
52000  done
53000  done
54000  done
55000  done
56000  done
57000  done
58000  done
59000  done
60000  done
61000  done
62000  done
63000  done
64000  done
65000  done
66000  done
67000  done
68000  done
69000  done
70000  done
71000  done
72000  done
73000  done
74000  done
75000  done
76000  done
77000  done
78000  done
79000  done
80000  done
81000  done
82000  done
83000  done
84000